In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Installing the required modules
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install pyfluidsynth pretty_midi

!pip install magenta
!pip install note_seq
!pip install tensorflow

Selecting previously unselected package fluid-soundfont-gm.
(Reading database ... 160815 files and directories currently installed.)
Preparing to unpack .../fluid-soundfont-gm_3.1-5.1_all.deb ...
Unpacking fluid-soundfont-gm (3.1-5.1) ...
Selecting previously unselected package libfluidsynth1:amd64.
Preparing to unpack .../libfluidsynth1_1.1.9-1_amd64.deb ...
Unpacking libfluidsynth1:amd64 (1.1.9-1) ...
Setting up fluid-soundfont-gm (3.1-5.1) ...
Setting up libfluidsynth1:amd64 (1.1.9-1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.2) ...
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

     |████████████████████████████████| 5.6MB 5.7MB/s 
     |████████████████████████████████| 51kB 6.2MB/s 
  Created wheel for pretty-midi: filename=pretty_midi-0.2.9-cp37-none-any.whl size=5591958 sha256=ac8fe615d38a263f453221bf6bda0dcbc9172e3204bcd4f28f00218346b6210b
  Stored in directory: /root/.cache/pip/wheels/4c/a1/c6/b5697841

In [3]:
#Source (code directly copy pasted from): note_seq/drums_encoder_decoder

# Default list of 9 drum types, where each type is represented by a list of
# MIDI pitches for drum sounds belonging to that type. This default list
# attempts to map all GM1 and GM2 drums onto a much smaller standard drum kit
# based on drum sound and function.
DEFAULT_DRUM_TYPE_PITCHES = [
    # kick drum
    [36, 35],

    # snare drum
    [38, 27, 28, 31, 32, 33, 34, 37, 39, 40, 56, 65, 66, 75, 85],

    # closed hi-hat
    [42, 44, 54, 68, 69, 70, 71, 73, 78, 80, 22],

    # open hi-hat
    [46, 67, 72, 74, 79, 81, 26],

    # low tom
    [45, 29, 41, 43, 61, 64, 84],

    # mid tom
    [48, 47, 60, 63, 77, 86, 87],

    # high tom
    [50, 30, 62, 76, 83],

    # crash cymbal
    [49, 52, 55, 57, 58],

    # ride cymbal
    [51, 53, 59, 82]
] 


#Source (code directly copy pasted from): magenta/models/music_vae/data.py

# 9 classes: kick, snare, closed_hh, open_hh, low_tom, mid_tom, hi_tom, crash,
# ride
REDUCED_DRUM_PITCH_CLASSES = DEFAULT_DRUM_TYPE_PITCHES
# 61 classes: full General MIDI set
FULL_DRUM_PITCH_CLASSES = [
    [p] for p in  # pylint:disable=g-complex-comprehension
    [36, 35, 38, 27, 28, 31, 32, 33, 34, 37, 39, 40, 56, 65, 66, 75, 85, 42, 44,
     54, 68, 69, 70, 71, 73, 78, 80, 46, 67, 72, 74, 79, 81, 45, 29, 41, 61, 64,
     84, 48, 47, 60, 63, 77, 86, 87, 50, 30, 43, 62, 76, 83, 49, 55, 57, 58, 51,
     52, 53, 59, 82]
]
ROLAND_DRUM_PITCH_CLASSES = [
    # kick drum
    [36],
    # snare drum
    [38, 37, 40],
    # closed hi-hat
    [42, 22, 44],
    # open hi-hat
    [46, 26],
    # low tom
    [43, 58],
    # mid tom
    [47, 45],
    # high tom
    [50, 48],
    # crash cymbal
    [49, 52, 55, 57],
    # ride cymbal
    [51, 53, 59]
]

DUALISED_PITCH_CLASSES = [
                          [74],
                          [60]
]

instruments = ["kick","snare","hat_closed","hat_open","tom_low","tom_mid","tom_high","cymbal_crash","cymbal_ride"]

In [4]:
from magenta.models.music_vae import data
import note_seq
from note_seq import midi_io
import os
import pandas as pd
import numpy as np

grooVAE_converter=data.GrooveConverter(
        split_bars=2, steps_per_quarter=4, quarters_per_bar=4,
        max_tensors_per_notesequence=20, humanize=True,
        pitch_classes=ROLAND_DRUM_PITCH_CLASSES,
        inference_pitch_classes=REDUCED_DRUM_PITCH_CLASSES)

dua_converter=data.GrooveConverter(
        split_bars=2, steps_per_quarter=4, quarters_per_bar=4,
        max_tensors_per_notesequence=20, humanize=True,
        pitch_classes=DUALISED_PITCH_CLASSES,
        inference_pitch_classes=DUALISED_PITCH_CLASSES)

def convert_midi_path_list_to_hvo_list(list_of_midi_path, dua_converter):
  hvo_list = []
  for midi_path in list_of_midi_path:
    dua_note_seq = midi_io.midi_file_to_note_sequence(midi_path)
    dua_hvo = dua_converter.to_tensors(dua_note_seq).outputs
    hvo_list.append(dua_hvo)

  return hvo_list

def cosine_sim(loop, performance_loops):
    # loop shape:  [32, 27]
    # performance_loops --> list of n elements each have a shape of [32, 27]
    
    a = loop.flatten()
    similarities = np.array([])
    for performance_loop in performance_loops:  
        performance_loop.shape
        b = performance_loop.flatten()
        if b.shape == a.shape: # Ignore shorter loops
            cosine_sim_for_loop = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
            similarities=np.append(similarities, cosine_sim_for_loop)
    return similarities
    
    
def get_self_cosine_similarity(performance_loops):
    self_sim_matrix = np.array([])
    for ix, loop in enumerate(performance_loops):
        if loop.shape[0] == 32:
            sim = cosine_sim(loop, performance_loops)
            if self_sim_matrix.shape[0]>0:
                self_sim_matrix = np.vstack((self_sim_matrix, sim))
            else: 
                self_sim_matrix = sim
    return self_sim_matrix

/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


# Restructuring

In [68]:
PATH = '/content/drive/MyDrive/Master thesis/ableton sessions/pau A1 Project'

import os
import pandas as pd
import numpy as np

session0_df = pd.read_csv(os.path.join(PATH, 'pauA1.csv'))
sorted_df_0 = session0_df.sort_values(by=['index'])
ori_list = sorted_df_0['grooVAE_name'].tolist()
dua_list = sorted_df_0['dualised_name'].tolist()

In [69]:
df_one_hour_session = pd.read_csv(os.path.join(PATH, 'pauA1.csv'))
df_one_hour_session = df_one_hour_session.drop_duplicates()
df_one_hour_session

session  track  ... Syncopation   style
0    pauA1      0  ...          41    soul
1    pauA1      1  ...          29    rock
2    pauA1      2  ...          29  reggae
3    pauA1      3  ...          30    rock
4    pauA1      4  ...          11    jazz
..     ...    ...  ...         ...     ...
64   pauA1     64  ...          28  hiphop
65   pauA1     65  ...           9    rock
66   pauA1     66  ...          19  hiphop
67   pauA1     67  ...          48   latin
68   pauA1     68  ...           0    rock

[69 rows x 9 columns]

In [70]:
MIDI_PATH = '/content/drive/MyDrive/Master thesis/ableton sessions/pau A1 Project/data'
name = sorted_df_0[sorted_df_0['index'] == df_one_hour_session.iloc[5]['index']].iloc[0].grooVAE_name
os.path.isfile(os.path.join(MIDI_PATH, name))

True

In [52]:
sorted_df_0[sorted_df_0['index'] == df_one_hour_session.iloc[0]['index']]

session  track  ... Syncopation style
0      B1      0  ...          19  rock

[1 rows x 9 columns]

In [71]:
import shutil
NEW_FOLDER_PATH = '/content/drive/MyDrive/Master thesis/ableton sessions/pau A1 Project/A1'
os.mkdir(NEW_FOLDER_PATH)

for j in range(len(df_one_hour_session)):
  track_name = df_one_hour_session.iloc[j].original_file_location.split('\\')[-1].split('.')[0]
  print('track no.', j)
  print(track_name)

  new_track_folder = os.path.join(NEW_FOLDER_PATH, track_name)
  if not os.path.isdir(new_track_folder):
    os.mkdir(new_track_folder)

  tmp_df = sorted_df_0[sorted_df_0['index'] == df_one_hour_session.iloc[j]['index']]
  for i in range(1):
    
    source = os.path.join(MIDI_PATH,tmp_df.iloc[i].dualised_name)
    print(source)
    target = os.path.join(new_track_folder, 'complex.mid')
    shutil.copyfile(source, target)
    if i == 0:
      source = os.path.join(MIDI_PATH,tmp_df.iloc[i].grooVAE_name)
      target = os.path.join(new_track_folder, 'original.mid')
      shutil.copyfile(source, target)
  print('\n')

track no. 0
drummer8-eval_session-3_soul-groove3_86_beat_4-4_best_2bar_segment_10
/content/drive/MyDrive/Master thesis/ableton sessions/pau A1 Project/data/pauA1_track00_drummer8-eval_session-3_soul-groove3_86_beat_4-4_best_2bar_segment_10_dualised.mid


track no. 1
drummer2-session2-8_rock_130_beat_4-4_best_2bar_segment_15
/content/drive/MyDrive/Master thesis/ableton sessions/pau A1 Project/data/pauA1_track01_drummer2-session2-8_rock_130_beat_4-4_best_2bar_segment_15_dualised.mid


track no. 2
drummer5-session2-5_reggae_126_beat_4-4_best_2bar_segment_32
/content/drive/MyDrive/Master thesis/ableton sessions/pau A1 Project/data/pauA1_track02_drummer5-session2-5_reggae_126_beat_4-4_best_2bar_segment_32_dualised.mid


track no. 3
drummer7-session2-45_rock_135_beat_4-4_best_2bar_segment_11
/content/drive/MyDrive/Master thesis/ableton sessions/pau A1 Project/data/pauA1_track03_drummer7-session2-45_rock_135_beat_4-4_best_2bar_segment_11_dualised.mid


track no. 4
drummer1-session3-2_jazz-swi

In [ ]:
sorted_df_0[sorted_df_0['index'] == df_one_hour_session.iloc[5]['index']]

session  track  ... Syncopation   style
69  repetition3     69  ...           0  reggae
5   repetition3      5  ...           0  reggae
62  repetition3     62  ...           0  reggae

[3 rows x 9 columns]

# Testing HVO

In [ ]:
MIDI_PATH = '/content/drive/MyDrive/Master thesis/ableton sessions/genis repetition Project/data'

ori_hvo_list = []
dua_hvo_list = []

for i in range(len(ori_list)):
  ori_note_seq = midi_io.midi_file_to_note_sequence(os.path.join(MIDI_PATH, ori_list[i]))
  dua_note_seq = midi_io.midi_file_to_note_sequence(os.path.join(MIDI_PATH, dua_list[i]))

  ori_hvo = grooVAE_converter.to_tensors(ori_note_seq).outputs
  dua_hvo = dua_converter.to_tensors(dua_note_seq).outputs

  ori_hvo_list.append(ori_hvo[0])
  dua_hvo_list.append(dua_hvo[0])

In [ ]:
dua_note_seq_0 = dua_converter.from_tensors(dua_hvo_list[0])
dua_note_seq_1 = dua_converter.from_tensors(dua_hvo_list[1])
dua_note_seq_2 = oVAE_converter.from_tensors(dua_hvo_list[2])

note_seq.plot_sequence(dua_note_seq_0)
note_seq.plot_sequence(dua_note_seq_1)
note_seq.plot_sequence(dua_note_seq_2)


IndexError: ignored

In [ ]:
dua_hvo_list[0].shape

(32, 6)

In [ ]:
import random
import note_seq
from note_seq import midi_io

MIDI_PATH = '/content/drive/MyDrive/Master thesis/ableton sessions/genis repetition Project/data'

i = random.randint(0, len(ori_list))


print('original file: ', ori_list[i])
print('dualised file: ', dua_list[i])

ori_note_seq = midi_io.midi_file_to_note_sequence(os.path.join(MIDI_PATH, ori_list[i]))
dua_note_seq = midi_io.midi_file_to_note_sequence(os.path.join(MIDI_PATH, dua_list[i]))

note_seq.plot_sequence(ori_note_seq)
note_seq.plot_sequence(dua_note_seq)

original file:  repetition0_track41_drummer7-eval_session-5_funk-groove5_84_beat_4-4_best_2bar_segment_9_original.mid
dualised file:  repetition0_track41_drummer7-eval_session-5_funk-groove5_84_beat_4-4_best_2bar_segment_9_dualised.mid


In [ ]:
ori_hvo = grooVAE_converter.to_tensors(ori_note_seq).outputs
print(ori_hvo[0].shape)
dua_hvo = dua_converter.to_tensors(dua_note_seq).outputs
print(dua_hvo[0].shape)

(32, 27)
(32, 6)


In [ ]:
test_decode = dua_converter.from_tensors(dua_hvo)
type(test_decode[0])

note_seq.protobuf.music_pb2.NoteSequence

In [ ]:
note_seq.plot_sequence(dua_note_seq)
note_seq.plot_sequence(test_decode[0])